## 1) Import libraries

In [1]:
import pandas as pd, datetime, gc
import folium
import geopandas as gpd
import matplotlib.pyplot as plt

## 2) Read in Boston Census 2020 Block Groups GeoJSON Data from Analyze Boston

In [2]:
bn_url = "https://data.boston.gov/dataset/c478b600-3e3e-46fd-9f57-da89459e9928/resource/98201cf0-8aa9-4751-a34d-4d45191a3456/download/census2020_blockgroups.geojson"
bcb = gpd.read_file(bn_url)
pd.set_option('display.max_columns',None)
bcb.head(1)

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLKGRPCE20,GEOID20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_STAr,Shape_STLe,geometry
0,0,1,25,025,040600,1,250250406001,Block Group 1,G5030,S,1265377,413598,+42.3833695,-071.0707743,1.807118e+07,29256.866068,"POLYGON ((769378.692 2964626.314, 769383.713 2..."


In [3]:
# Set Coordinate Reference System to EPSG: 4326
bcb = bcb.to_crs(epsg=4326)

In [4]:
# Trim geodatframe down to only the fields we need
bcb = bcb[['GEOID20','NAMELSAD20','geometry']]

In [5]:
bcb['GEOID20'] = bcb['GEOID20'].astype(int)

## 3) Read in census block groups closure rate prediction scores

In [6]:
pred_url = "https://raw.githubusercontent.com/georgetown-analytics/boston-311/main/03%20output/predictions.csv"
pred = pd.read_csv(pred_url, dtype={'GEOID20':'int'})
del pred['Unnamed: 0']
pred.head()

,GEOID20,predictions
0,250250001011,0.650000
1,250250001012,0.674419
2,250250001021,0.642384
3,250250001022,0.353712
4,250250002011,0.651515


## 4) Merge prediction scores with Boston census block group geodataframe

In [7]:
bcb_pred = pd.merge(left=bcb,right=pred,how='left',left_on='GEOID20',right_on='GEOID20')

In [8]:
# Three null values are primarily reflected on the islands off the mainland of Boston
bcb_pred.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 581 entries, 0 to 580
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   GEOID20      581 non-null    int64   
 1   NAMELSAD20   581 non-null    object  
 2   geometry     581 non-null    geometry
 3   predictions  579 non-null    float64 
dtypes: float64(1), geometry(1), int64(1), object(1)
memory usage: 22.7+ KB


In [9]:
bcb_pred = bcb_pred.rename(columns={'GEOID20':'Geographic Record Identifier:','predictions':'Predicted Closure Rate:','NAMELSAD20':'Block Group Number:'})

In [10]:
m = bcb_pred.explore(column='Predicted Closure Rate:', name='311 Closure Rate', tiles='CartoDB positron')
folium.LayerControl().add_to(m)
m

In [ ]:
m.save("Boston 311 Closure Rate Predictions.html")